In [ ]:
import gymnasium as gym
import numpy as np
import random

In [ ]:
learning_rate = 0.1
discount = 0.95

epsilon = 0.5
decay = 0.98

num_episodes = 10000

render_delay = 1000

render = False

scenario = 'MountainCar-v0'

if render:
    env = gym.make(scenario, render_mode='human')

else:
    env = gym.make(scenario)
env.reset()


In [ ]:
num_actions = env.action_space.n
print(num_actions)

discrete_obs_space_size = [10] * len(env.observation_space.high)

discrete_obs_space_window_size = (env.observation_space.high - env.observation_space.low) / discrete_obs_space_size
print(discrete_obs_space_size)
q_table = np.random.uniform(low = -2, high = 0, size = (discrete_obs_space_size + [num_actions]))


In [ ]:
def get_disc_state(state):
    discrete_state = (state - env.observation_space.low) / discrete_obs_space_window_size
    return tuple(discrete_state.astype(int))


In [ ]:

for episode in range(num_episodes):

    discrete_state = get_disc_state(env.reset()[0])
    
    done = False

    while not done:


        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, num_actions-1)
        else:
            action = np.argmax(q_table[discrete_state])

        epsilon *= decay


        new_state, reward, termination, truncation, _ = env.step(action)

        done = termination

        new_discrete_state = get_disc_state(new_state)

        

        if not done:
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action, )]

            new_q = (1 - learning_rate) * current_q + learning_rate * (reward + discount * max_future_q)

            q_table[discrete_state + (action, )] = new_q

        else:
            print(f'Made it on episode {episode}')


        discrete_state = new_discrete_state
    if (episode + 1) % render_delay == 0:
        env = gym.make(scenario, render_mode='human')
        env.reset()
    else:
        env = gym.make(scenario)
        env.reset()


